# Notebook 3 : SQL

In [ ]:
# Décommenter la ligne suivante pour installer ibis
# %pip install 'ibis-framework[sqlite]'

In [ ]:
import sqlite3

import pandas as pd
import ibis

from ibis import _

ibis.options.interactive = True

query_tables = "SELECT name FROM sqlite_master WHERE type='table'"

## STAR

Nous considérons les données des stations de vélos en libre service [STAR](https://www.star.fr/) de Rennes Métropole. Une copie de la base SQLite est disponible dans le fichier `star.db`. Nous utilisons d'abord Pandas pour répondre aux questions, puis Ibis.

1. Se connecter à la base de données et afficher la liste des tables à l'aide de la fonction `read_sql` de Pandas et de la requête `query_tables`.

In [ ]:
con = sqlite3.connect("data/star.db")
tables = pd.read_sql(query_tables, con)
tables

2. Récupérer le contenu de la table `Etat` dans un dataframe et afficher la liste des variables disponibles. Même question pour la table `Topologie`.

In [ ]:
etat_df = pd.read_sql("SELECT * FROM Etat", con)
etat_df.dtypes

In [ ]:
topologie_df = pd.read_sql("SELECT * FROM Topologie", con)
topologie_df.dtypes

3. Sélectionner l'identifiant `id`, le nom `nom` et l'identifiant de la station la plus proche `id_proche_1` depuis la table `Topologie`.

In [ ]:
topologie_df.filter(items=["id", "nom", "id_proche_1"])

4. Faire une jointure sur la table précédente pour créer une table qui contient la liste des stations avec l'identifiant, le nom et le nom de la station la plus proche associée à l'identifiant `id_proche_1`. Les variables utilisées comme clés sont différents, penser à utiliser les arguments `left_on` et `right_on` de la méthode `merge`.

In [ ]:
(
    topologie_df
    .merge(etat_df, how="left", left_on="id_proche_1", right_on="id")
    .filter(items=["id_x", "nom_x", "nom_y"])
    .rename(columns={"id_x": "id", "nom_x": "nom", "nom_y": "nom_proche_1"})
)

5. Ajouter à la table précédente la distance entre la station et la station la plus proche.

In [ ]:
(
    topologie_df
    .merge(etat_df, how="left", left_on="id_proche_1", right_on="id")
    .assign(
        # La méthode assign fonctionne aussi avec une fonction anonyme
        distance=lambda row: (
            (row.latitude_x - row.latitude_y)**2
            + (row.longitude_x - row.longitude_y)**2
        )**0.5
    )
    .filter(items=["id_x", "nom_x", "nom_y", "distance"])
    .rename(columns={"id_x": "id", "nom_x": "nom", "nom_y": "nom_proche_1"})
)

6. Créer une table avec le nom des trois stations les plus proches du point GPS *(48.1179151,-1.7028661)* classées par ordre de distance et le nombre de vélos disponibles dans ces stations.

In [ ]:
(
    etat_df
    .assign(
        distance=(
            (48.1179151 - etat_df.latitude)**2
            + (-1.7028661 - etat_df.longitude)**2
        )**0.5
    )
    .nsmallest(3, "distance") # Plus simple que sort_values et head
    .filter(items=["nom", "distance", "velos_disponibles"])
)

7. Reprendre les questions précédentes en utilisant le module `ibis`. Pour les jointures, utiliser la méthode `left_join`.

In [ ]:
# Question 1
con_ibis = ibis.sqlite.connect("data/star.db")
con_ibis.tables

In [ ]:
# Question 2
etat_table = con_ibis.table("Etat")
print("*** Etat ***")
print(etat_table.columns)

topologie_table = con_ibis.table("Topologie")
print("*** Topologie ***")
print(topologie_table.columns)

In [ ]:
# Question 3
topologie_table.select("id", "nom", "id_proche_1")

In [ ]:
# Question 4
(
    topologie_table
    .left_join(
        etat_table,
        topologie_table.id_proche_1 == etat_table.id
    )
    .select("id", "nom", "nom_right")
    .rename(nom_proche_1="nom_right") # Renommer une colonne
)
    

In [ ]:
# Question 5
(
    topologie_table
    .left_join(
        etat_table,
        topologie_table.id_proche_1 == etat_table.id
    )
    .mutate(
        # Utilisation de _
        d_lat = _.latitude - _.latitude_right,
        d_lon = _.longitude - _.longitude_right
    )
    .mutate(
        distance = (_.d_lat * _.d_lat + _.d_lon * _.d_lon)**0.5
    )
    .select("id", "nom", "nom_right", "distance")
    .rename(nom_proche_1="nom_right")
)

In [ ]:
# Question 6
(
    etat_table
    .mutate(
        d_lat = 48.1179151 - etat_table.latitude,
        d_lon = -1.7028661 - etat_table.longitude
    )
    .mutate(
        # Utilisation de _
        distance = (_.d_lat * _.d_lat + _.d_lon * _.d_lon)**0.5
    )
    .order_by("distance")
    .limit(3)
    .select("nom", "distance", "velos_disponibles")
)

8. (*Bonus*) Écrire des requêtes SQL pour obtenir les résultats demandés dans les questions 3 à 6. La fonction `to_sql` pourra être utilisée pour de l'aide.

In [ ]:
# Question 3
query = """
SELECT id, nom, id_proche_1
FROM Topologie
"""

pd.read_sql(query, con)

In [ ]:
# Question 4
query = """
SELECT
    Topologie.id,
    Topologie.nom,
    Etat.nom AS nom_proche_1
FROM Topologie
LEFT JOIN Etat ON Topologie.id_proche_1 = Etat.id
"""

pd.read_sql(query, con)

In [ ]:
# Question 5
query = """
SELECT
    Topologie.id,
    Topologie.nom,
    Etat.nom AS nom_proche_1,
    POWER(
        POWER(
            Etat.latitude - Topologie.latitude,
            2
        )
        + POWER(
            Etat.longitude - Topologie.longitude,
            2
        ),
        0.5
    ) AS distance
FROM Topologie
LEFT JOIN Etat ON Topologie.id_proche_1 = Etat.id
"""

pd.read_sql(query, con)

In [ ]:
# Question 6
query = """
SELECT
    nom,
    POWER(
        POWER(
            48.1179151 - Etat.latitude,
            2
        )
        + POWER(
            -1.7028661 - Etat.longitude,
            2
        ),
        0.5
    ) AS distance,
    velos_disponibles
FROM Etat
ORDER BY distance
LIMIT 3
"""

pd.read_sql(query, con)

## Musique

Le dépôt GitHub [lerocha/chinook-database](https://github.com/lerocha/chinook-database) met à disposition des bases de données de bibliothèques musicales. Une copie de la base SQLite est disponible dans le fichier `chinook.db`.

1. Utiliser le module `ibis` pour vous connecter à la base de données et explorer les tables formant le jeu de données pour le découvrir. En particulier, remarquer comment les tables `Playlist`, `PlaylistTrack` et `Track` sont liées entre elles.

In [ ]:
con_chinook = ibis.sqlite.connect("data/chinook.db")
con_chinook.tables

In [ ]:
# Les entrées de la table Playlist sont identifiées par l'identifiant PlaylistId
playlist = con_chinook.table("Playlist")
playlist

In [ ]:
# Les entrées de la table Track sont identifiées par l'identifiant TrackId
track = con_chinook.table("Track")
track

In [ ]:
# La table PlaylistTrack fait le lien entre Playlist et Track grâce aux identifiants PlaylistId et TrackId.
# Pour chaque PlaylistId, les pistes sont données par les TrackId associés dans la table PlaylistTrack.
# Il s'agit d'une organisation classique des bases de données relationnelles.
playlist_track = con_chinook.table("PlaylistTrack")
playlist_track

2. Quelles sont les playlists qui contiennent le plus de pistes ?

In [ ]:
(
    playlist_track
    .group_by("PlaylistId")
    .aggregate(n_track=playlist_track.count())
    .left_join(
        # Jointure pour récupérer le nom de la playlist
        playlist, _.PlaylistId == playlist.PlaylistId
    )
    .select("PlaylistId", "Name", "n_track")
    .order_by(
        # Tri décroissant avec Ibis
        ibis.desc("n_track")
    )
)

3. Construire une table contenant les informations suivantes sur la playlist `Classical` : le titre de chaque piste ainsi que le titre de l'album dont cette piste est tirée.

In [ ]:
# Table Album
album = con_chinook.table("Album")

(
    playlist_track
    .left_join(playlist, playlist_track.PlaylistId == playlist.PlaylistId)
    .rename(PlaylistName="Name")
    .filter(_.PlaylistName == "Classical")
    .left_join(track, _.TrackId == track.TrackId)
    .left_join(album, _.AlbumId == album.AlbumId)
    .select("Name", "Title")
)

4. (*Bonus*) Écrire une requête SQL donnant le résultat de la question précédente. La fonction `to_sql` pourra être utilisée pour de l'aide.

In [ ]:
con = sqlite3.connect("data/chinook.db")

query = """
SELECT
    TrackNames.Name,
    Album.Title
FROM (
    SELECT
        Track.Name,
        Track.AlbumId
    FROM (
        SELECT PlaylistTrack.TrackId
        FROM PlaylistTrack
        LEFT JOIN Playlist
        ON PlaylistTrack.PlaylistId = Playlist.PlaylistId
        WHERE Playlist.Name = 'Classical'
    ) AS TrackIds
    LEFT JOIN Track
    ON TrackIds.TrackId = Track.TrackId
) AS TrackNames
LEFT JOIN Album
ON TrackNames.AlbumId = Album.AlbumId
"""

pd.read_sql(query, con)